In [1]:
!python --version

Python 3.11.13


In [ ]:
%pip install -q -U "transformers>=4.51.0" "accelerate>=1.0.0" "trl>=0.9.6" "datasets>=2.18.0"
%pip install -q unsloth bitsandbytes
%pip install -q chromadb sentence-transformers sacrebleu evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.4/517.4 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
import torch
from unsloth import FastLanguageModel
import chromadb
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
import torch
from datasets import Dataset
from trl import SFTTrainer, SFTConfig

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-12-16 18:31:09.981184: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765909870.141376      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765909870.189849      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
import os
import torch
from unsloth import FastLanguageModel

DATA_DIR = "/kaggle/input/vlst-medical-mt-cleaned"
TRAIN_EN_PATH = os.path.join(DATA_DIR, "train_ratio_filtered.en.txt")
TRAIN_VI_PATH = os.path.join(DATA_DIR, "train_ratio_filtered.vi.txt")
TEST_EN_PATH = os.path.join(DATA_DIR, "clean_public_test.en.txt")
TEST_VI_PATH = os.path.join(DATA_DIR, "clean_public_test.vi.txt")

MAX_SAMPLES = None  
MAX_TEST_SAMPLES = None 

MODEL_NAME = "unsloth/Qwen3-0.6B-unsloth-bnb-4bit" 
MAX_SEQ_LENGTH = 4096 
TRAIN_MAX_LEN = 512 
SEED = 3407

CHROMA_DB_DIR = "/kaggle/working/chroma_medical_en_vi" 
N_RETRIEVAL = 3 

torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print("="*60)
print("CONFIGURATION")
print("="*60)
print(f"Model: {MODEL_NAME}")
print(f"Max Seq Length: {MAX_SEQ_LENGTH}")
print(f"Train Max Length: {TRAIN_MAX_LEN}")
print(f"RAG Examples: {N_RETRIEVAL}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print("="*60)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    load_in_4bit=True,
    load_in_8bit=False,
    full_finetuning=False,
)

print("\n✓ Model loaded successfully!")

CONFIGURATION
Model: unsloth/Qwen3-0.6B-unsloth-bnb-4bit
Max Seq Length: 4096
Train Max Length: 512
RAG Examples: 3
CUDA: True
GPU: Tesla T4
==((====))==  Unsloth 2025.12.5: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/576M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]


✓ Model loaded successfully!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,
    lora_dropout=0.0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=SEED,
    use_rslora=False,
    loftq_config=None,
)

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"✓ LoRA adapters added")
print(f"✓ Trainable parameters: {trainable:,}")
print(f"✓ Model on device: {next(model.parameters()).device}")

Unsloth 2025.12.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


✓ LoRA adapters added
✓ Trainable parameters: 20,185,088
✓ Model on device: cuda:0


In [ ]:
def load_parallel_data(en_path, vi_path, max_samples=None):
    """Load English-Vietnamese parallel data"""
    with open(en_path, "r", encoding="utf-8") as f:
        en_lines = [line.strip() for line in f]
    
    with open(vi_path, "r", encoding="utf-8") as f:
        vi_lines = [line.strip() for line in f]
    
    assert len(en_lines) == len(vi_lines), "Mismatch: EN and VI lines!"
    
    if max_samples:
        en_lines = en_lines[:max_samples]
        vi_lines = vi_lines[:max_samples]
    
    data = [(en, vi) for en, vi in zip(en_lines, vi_lines) if en and vi]
    
    print(f"Loaded {len(data):,} parallel pairs")
    return data

print("Loading training data...")
train_pairs = load_parallel_data(TRAIN_EN_PATH, TRAIN_VI_PATH, MAX_SAMPLES)

print("Loading test data...")
test_pairs = load_parallel_data(TEST_EN_PATH, TEST_VI_PATH, MAX_TEST_SAMPLES)

print(f"\nSample:")
print(f"  EN: {train_pairs[0][0]}")
print(f"  VI: {train_pairs[0][1]}")

Loading training data...
Loaded 347,379 parallel pairs
Loading test data...
Loaded 3,000 parallel pairs

Sample:
  EN: To evaluate clinical, subclinical symptoms of patients with otitis media with effusion and V.a at otorhinolaryngology department – Thai Nguyen national hospital
  VI: Nghiên cứu đặc điểm lâm sàng, cận lâm sàng bệnh nhân viêm tai ứ dịch trên viêm V.A tại Khoa Tai mũi họng - Bệnh viện Trung ương Thái Nguyên


In [ ]:
from datasets import Dataset

SYSTEM_PROMPT = "You are an expert medical translator from English to Vietnamese."

def format_training_sample(en_text, vi_text):
    """Format EN-VI pair for training with Qwen3 chat template"""
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"Translate to Vietnamese: {en_text}"},
        {"role": "assistant", "content": vi_text}
    ]
    
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False,
        enable_thinking=False,
    )
    return text

print("Preparing training data with filtering...")
train_texts = []
skipped = 0

for en, vi in tqdm(train_pairs, desc="Formatting"):
    text = format_training_sample(en, vi)
    token_count = len(tokenizer.encode(text, add_special_tokens=True))
    
    if token_count <= TRAIN_MAX_LEN:
        train_texts.append(text)
    else:
        skipped += 1

print(f"\nKept: {len(train_texts):,} samples")
print(f"Skipped: {skipped} samples (>{TRAIN_MAX_LEN} tokens)")

train_dataset = Dataset.from_dict({"text": train_texts})
train_dataset = train_dataset.shuffle(seed=SEED)

print(f"\nSample formatted text:")
print(train_dataset[0]["text"][:])

Preparing training data with filtering...


Formatting:   0%|          | 0/347379 [00:00<?, ?it/s]


Kept: 347,351 samples
Skipped: 28 samples (>512 tokens)

Sample formatted text:
<|im_start|>system
You are an expert medical translator from English to Vietnamese.<|im_end|>
<|im_start|>user
Translate to Vietnamese: Conclusion: Combined spinal and epidural anesthesia provided effective anesthesia for hip replacement surgery with high success rates in one vertebrae technique, shorter duration of technique, lower hypotension.<|im_end|>
<|im_start|>assistant
<think>

</think>

Kết luận: Gây tê tuỷ sống và ngoài màng cứng kết hợp có hiệu quả tốt trong phẫu thuật thay khớp háng với tỉ lệ thành công cao trong kỹ thuật một đốt sống, thời gian thực hiện kỹ thuật ngắn, tỉ lệ tụt huyết áp thấp.<|im_end|>



In [ ]:
import os, torch
from unsloth import FastLanguageModel
from peft import PeftModel
from transformers import AutoTokenizer

BASE_MODEL    = "unsloth/Qwen3-0.6B-unsloth-bnb-4bit"
MODEL_OUTPUT_DIR    = "/kaggle/input/lora32-en-vi-qwen3-0-6b-en-vi-medical-final/lora32-en-vi-qwen3-0.6b-en-vi-medical-final"
ADAPTER_DIR   = os.path.join(MODEL_OUTPUT_DIR, "lora_adapter")
TOKENIZER_DIR = os.path.join(MODEL_OUTPUT_DIR, "tokenizer")

assert os.path.isdir(ADAPTER_DIR), f"Missing: {ADAPTER_DIR}"
assert os.path.isdir(TOKENIZER_DIR), f"Missing: {TOKENIZER_DIR}"

model, _ = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL,
    max_seq_length=4096,
    dtype=None,
    load_in_4bit=True,
)

tokenizer = AutoTokenizer.from_pretrained(
    TOKENIZER_DIR,
    use_fast=True,
    trust_remote_code=True,
)

model = PeftModel.from_pretrained(model, ADAPTER_DIR)


==((====))==  Unsloth 2025.12.5: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
from unsloth import FastLanguageModel

print(SYSTEM_PROMPT)
FastLanguageModel.for_inference(model)

def translate_en_to_vi(en_text, max_new_tokens=512):
    """
    Translate English to Vietnamese directly (no RAG/few-shot)
    
    Args:
        en_text: English input
        max_new_tokens: Max output length
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"Translate to Vietnamese: {en_text}"}
    ]
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=MAX_SEQ_LENGTH)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    generated_ids = outputs[0][inputs['input_ids'].shape[1]:]
    translation = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
    
    return translation

print("Testing direct translation (EN → VI, no RAG):\n")
for i in range(3):
    print(f"--- Example {i+1} ---")
    en_src = test_pairs[i][0]
    vi_ref = test_pairs[i][1]
    print(f"EN:   {en_src}")
    vi_pred = translate_en_to_vi(en_src)
    print(f"PRED: {vi_pred}")
    print(f"REF:  {vi_ref}")
    print()

You are an expert medical translator from English to Vietnamese.
Testing direct translation (EN → VI, no RAG):

--- Example 1 ---
EN:   Knowledge, practices in public health service utilization among health insurance card’s holders and influencing factors in Vientiane, Lao
PRED: Kiến thức, thực hành sử dụng dịch vụ y tế của người sử dụng thẻ bảo hiểm y tế và các yếu tố ảnh hưởng tại Vientiane, Lào
REF:  Thực trạng kiến thức và thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố ảnh hưởng tại tỉnh Viêng Chăn, CHDCND Lào, năm 2017

--- Example 2 ---
EN:   Describe knowledge, practices in public health service utilization among health insurance card's holders and influencing factors in Vientiane, Lao PDR, 2017.
PRED: Mô tả kiến thức, thực hành sử dụng dịch vụ y tế của người sử dụng thẻ bảo hiểm y tế và các yếu tố ảnh hưởng tại Vientiane, Lào PĐ, năm 2017.
REF:  Mô tả thực trạng kiến thức, thực hành của người có thẻ bảo hiểm 

In [ ]:
import evaluate
from tqdm.auto import tqdm

print(f"Translating {len(test_pairs):,} test samples (EN → VI, no RAG)...\n")

predictions = []
references = []

for en_text, vi_ref in tqdm(test_pairs, desc="Translating"):
    vi_pred = translate_en_to_vi(en_text)
    predictions.append(vi_pred)
    references.append(vi_ref)

metric = evaluate.load("sacrebleu")
bleu_result = metric.compute(
    predictions=predictions,
    references=[[ref] for ref in references]
)

print("\n" + "="*60)
print("EVALUATION RESULTS")
print("="*60)
print(f"BLEU Score: {bleu_result['score']:.2f}")
print(f"BLEU-1: {bleu_result['precisions'][0]:.2f}")
print(f"BLEU-2: {bleu_result['precisions'][1]:.2f}")
print(f"BLEU-3: {bleu_result['precisions'][2]:.2f}")
print(f"BLEU-4: {bleu_result['precisions'][3]:.2f}")
print("="*60)

print("\nSample predictions:\n")
for i in range(min(5, len(test_pairs))):
    print(f"Example {i+1}:")
    print(f"  EN:   {test_pairs[i][0][:80]}...")
    print(f"  REF:  {references[i][:80]}...")
    print(f"  PRED: {predictions[i][:80]}...")
    print()

Translating 3,000 test samples (EN → VI, no RAG)...



Translating:   0%|          | 0/3000 [00:00<?, ?it/s]


EVALUATION RESULTS
BLEU Score: 41.21
BLEU-1: 67.38
BLEU-2: 47.98
BLEU-3: 35.05
BLEU-4: 26.38

Sample predictions:

Example 1:
  EN:   Knowledge, practices in public health service utilization among health insurance...
  REF:  Thực trạng kiến thức và thực hành của người có thẻ bảo hiểm y tế trong sử dụng d...
  PRED: Kiến thức, thực hành sử dụng dịch vụ y tế của người sử dụng thẻ bảo hiểm y tế và...

Example 2:
  EN:   Describe knowledge, practices in public health service utilization among health ...
  REF:  Mô tả thực trạng kiến thức, thực hành của người có thẻ bảo hiểm y tế trong sử dụ...
  PRED: Mô tả kiến thức, thực hành sử dụng dịch vụ y tế của người sử dụng thẻ bảo hiểm y...

Example 3:
  EN:   Methodology: A cross sectional study was used among 928 adult health insurance c...
  REF:  Phương pháp: Thiết kế nghiên mô tả cắt ngang được thực hiện trên 928 người trưởn...
  PRED: Phương pháp nghiên cứu: Nghiên cứu cắt ngang mô tả trên 928 người dân có thẻ bảo...

Example 4:
  EN:   R

In [ ]:
import pandas as pd

results_df = pd.DataFrame({
    "english": [pair[0] for pair in test_pairs],
    "vietnamese_ref": references,
    "vietnamese_pred": predictions
})
OUTPUT_DIR=""
csv_path = os.path.join(OUTPUT_DIR, "test_predictions.csv")
results_df.to_csv(csv_path, index=False, encoding="utf-8")
print(f"✓ Saved detailed results: {csv_path}")

pred_path = os.path.join(OUTPUT_DIR, "predictions.vi.txt")
with open(pred_path, "w", encoding="utf-8") as f:
    for pred in predictions:
        f.write(pred + "\n")
print(f"✓ Saved predictions: {pred_path}")

summary_path = os.path.join(OUTPUT_DIR, "evaluation_summary.txt")
with open(summary_path, "w", encoding="utf-8") as f:
    f.write("VLSP 2025 Medical MT Evaluation\n")
    f.write("="*60 + "\n")
    f.write(f"Model: {MODEL_NAME}\n")
    f.write(f"Direction: EN → VI\n")
    f.write(f"Strategy: RAG + Few-shot ({N_RETRIEVAL} examples)\n")
    f.write(f"Training samples: {len(train_texts):,}\n")
    f.write(f"Test samples: {len(test_pairs):,}\n")
    f.write(f"\nBLEU Score: {bleu_result['score']:.2f}\n")
    f.write(f"BLEU-1: {bleu_result['precisions'][0]:.2f}\n")
    f.write(f"BLEU-2: {bleu_result['precisions'][1]:.2f}\n")
    f.write(f"BLEU-3: {bleu_result['precisions'][2]:.2f}\n")
    f.write(f"BLEU-4: {bleu_result['precisions'][3]:.2f}\n")
print(f"✓ Saved summary: {summary_path}")

print("\n✓ All results saved!")

✓ Saved detailed results: test_predictions.csv
✓ Saved predictions: predictions.vi.txt
✓ Saved summary: evaluation_summary.txt

✓ All results saved!
